In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures

In [3]:
# read in all the words
words = open('Datasets/names.txt', 'r').read().splitlines()
print(len(words))
print(max([len(w) for w in words]))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [4]:
# build the vocabulary of the characters and mappings to / from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
    
    X, Y = [], []
    
    for w in words:
    
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '------>', itos[ix])
            context = context[1:] + [ix] # crop and append
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


# ok boilerplaye done, now we get to the action

In [6]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')


In [7]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd),             generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5) 
b1 = torch.randn(n_hidden,                        generator=g) * 0.01
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0
# BatchNorm Parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*1.0

# Note: I am initialising many of these parameters in non-standard ways
# because sometimes initializing with e.g. all zeros could mask an incorrect
# implementation of the backward pass

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [8]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y

In [9]:
a = torch.Tensor([[1, 2, 3, 4],
                  [4, 3, 2, 1]])
a.sum(1, keepdim=True).shape

torch.Size([2, 1])

In [10]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into the vectors # 32x3x10
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors # 32 x 30
# Linear Layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-batch normalisation # 32 x 64
# BatchNorm Layer
bnmeani = (1 / n) * hprebn.sum(0, keepdim=True) # 1 x 64
bndiff = hprebn - bnmeani # 32 x 64
bndiff2 = bndiff**2 # 32 x 64
bnvar = (1/(n-1))*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction ( dividing by n - 1, not n) # 1 x 64
bnvar_inv = (bnvar + 1e-5) ** -0.5 # 1 x 64
bnraw = bndiff * bnvar_inv # 32 x 64
hpreact = bngain * bnraw + bnbias # 32 x 64
# Non-linearity
h = torch.tanh(hpreact) # hidden layer # 32 x 64
# Linear Layer 2
logits = h @ W2 + b2 # output layer # 32 x 27
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values # 32 x 1
norm_logits = logits - logit_maxes # subtract max for numerical stability # 32 x 27
counts = norm_logits.exp() # 32 x 27
counts_sum = counts.sum(1, keepdim=True) # 32 x 1
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be a bit exact ... # 32 x 1
probs = counts * counts_sum_inv # 32 x 27
logprobs = probs.log() # 32 x 27
loss = -logprobs[range(n), Yb].mean() # 1 x 1

# PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.2872, grad_fn=<NegBackward0>)

In [11]:
counts_sum_inv.shape

torch.Size([32, 1])

In [29]:
'''
a1 a2
a3 a4

b1 b2 b3
b4 b5 b6

a1 (b2b6 - b5 b3) + a2 (b1 b5


'''
print(b2.shape)

torch.Size([27])


In [54]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs) # 32 x 27
dlogprobs[range(n), Yb] = -1.0/n # 32 x 27


dprobs = (1.0 / probs) * dlogprobs # 32 x 27

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True) # 32 x 1

dcounts = counts_sum_inv * dprobs # 32 x 27

dcounts_sum = -1/(counts_sum**2) * dcounts_sum_inv # 32 x 1

dcounts += torch.ones_like(dcounts) * dcounts_sum # 32 x 27

dnorm_logits = counts * dcounts # 32 x 27

dlogits = dnorm_logits.clone() # 32 x 27

dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True) # 32 x 1
# print(logits.shape, logit_maxes.expand(-1,27).shape, (logits == logit_maxes.expand(-1, 27)).to(torch.int))
dlogits += ((logits == logit_maxes.expand(-1, 27)).to(torch.int) * dlogit_maxes)

# h 32 x 64
# W2 64 x 27
# dlogits 32 x 27
# logits = h @ W2
# logits @ W2T = h # 32 x64
# hT @ logits = W2 # 64 x 27
# b2 32 x 27
dh = dlogits @ W2.t()

dW2 = h.t() @ dlogits

db2 = dlogits.sum(0)

dhpreact = (1.0 - h**2) * dh

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)

dbnraw = bngain * dhpreact

dbnbias = dhpreact.sum(0, keepdim=True)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

dbnvar = (-1/2 * (bnvar+1e-5)**(-3/2)) * dbnvar_inv

dbndiff2 = (1/(n-1))*torch.ones_like(dbnvar) * dbnvar

dbndiff = (2*bndiff) * dbndiff2
dbndiff += bnvar_inv * dbnraw

dbnmeani = (-1.0 * dbndiff).sum(0, keepdim=True)

dhprebn = ((1/n)*torch.ones_like(hprebn)) * dbnmeani
dhprebn += dbndiff

dembcat = dhprebn @ W1.T

dW1 = embcat.T @ dhprebn

db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        char = Xb[i, j]
        dC[char] += demb[i, j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)


logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [55]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.2871737480163574 diff: -4.76837158203125e-07


In [58]:
# backward pass

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 6.6356733441352844e-09


In [61]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased =True)) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(2.8610e-05, grad_fn=<MaxBackward1>)


In [64]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)
dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
cmp('hprebn', dhprebn, hprebn) # i can only get approximate to be true,my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 8.731149137020111e-11


In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass


In [ ]:
x = torch.randn(1000, 30)
w = torch.randn(30, 200) / (30**0.5)
y = x@w
print(x.mean(), x.std())
print(y.mean(), y.std())
plt.figure(figsize=(20, 5))
plt.subplot(121)
plt.hist(x.view(-1).tolist(), 50, density=True);
plt.subplot(122)
plt.hist(y.view(-1).tolist(), 50, density=True);

In [ ]:
# Same optimisation as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    # mini batch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y
    
    # forward pass
    emb = C[Xb] # Embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    hpreact = embcat@W1# + b1 # hidden layer pre-activation
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * ((hpreact - bnmeani) / bnstdi) + bnbias
    
    h = torch.tanh(hpreact) # hidden layer
    logits = h @W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update

    lr = 0.1 if i<100000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data  += -lr * p.grad

    # track stats
    if i%10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

In [ ]:
plt.plot(lossi)

In [ ]:
@torch.no_grad() # This decorator disables gradient tracking
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x] # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    hpreact = embcat@W1 # + b1 # hidden layer pre-activation
    # hpreact = bngain * ((hpreact - hpreact.mean(0, keepdim=True)) / (hpreact.std(0, keepdim=True))) + bnbias
    hpreact = bngain * ((hpreact - bnmean_running) / bnstd_running) + bnbias
    h = torch.tanh(hpreact) # (N, n_hidden)
    logits = h @ W2 + b2 #  (N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# sample from the model
g=torch.Generator().manual_seed(2147483647 + 10)
with torch.no_grad():
    for _ in range(20):
    
        out = []
        context = [0] * block_size # initialize with all ...
        while True:
            # forward pass the neural net
            emb = C[torch.tensor([context])] # (1, block_size, d)
            embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
            hpreact = embcat@W1 # + b1 # hidden layer pre-activation
            hpreact = bngain * ((hpreact - bnmean_running) / bnstd_running) + bnbias
            
            h = torch.tanh(hpreact) # hidden layer
            logits = h@W2 + b2 # output layer
    
            # h = torch.tanh(emb.view(1, -1) @ W1 + b1)
            # logits = h@W2 + b2
            probs = F.softmax(logits, dim=1)
            # sample from the distribution
            ix = torch.multinomial(probs, num_samples=1, generator=g).item()
            #shift the context window and track the samples
            context = context[1:] + [ix]
            out.append(ix)
            # if we sample the special '.' token, break
            if ix == 0:
                break
    
        print(''.join(itos[i] for i in out))